In [ ]:
import openreview
import icml

client = openreview.Client()

In [ ]:
# post the groups in the conference's path
icml_groups = openreview.tools.build_groups(icml.conference.id)
for group in icml_groups:
    try:
        existing_group = client.get_group(group.id)
    except openreview.OpenReviewException as e:
        posted_group = client.post_group(group)
        print(posted_group.id)

In [ ]:
# post the conference group with updated parameters
client.post_group(icml.conference)

### Features

- Reviewer and Area Chair recruitment
- Double-blind Paper Submission
- Area Chair and Reviewer bidding
- Areachair recommendations for reviewers
- Reviewer assignment
- Reviews and Comments
- Area chair and Program chair console(s)

### To Do Features
- Reviewer and area chair reviewer matching browser


### Reviewer and Area Chair recruitment

In [ ]:
# post reviewer groups that we can use to track those that were invited, 
# and those that accepted or declined the invitation.

icml.reviewers.members = [
    'redreviewer@openreview.net',
    'bluereviewer@openreview.net',
    'greenreviewer@openreview.net'
]

client.post_group(icml.reviewers)
client.post_group(icml.reviewers_invited)
client.post_group(icml.reviewers_declined)

In [ ]:
# do the same for area chairs.
icml.area_chairs.members = ['icmlareachair@openreview.net']

client.post_group(icml.area_chairs)
client.post_group(icml.area_chairs_invited)
client.post_group(icml.area_chairs_declined)

In [ ]:
# post the recruitment invitations
client.post_invitation(icml.recruit_reviewers)
client.post_invitation(icml.recruit_area_chairs)

In [ ]:
# define some potential reviewers and area chairs
rev_email = 'icmlreviewer@openreview.net'
rev_firstname = 'ICML Reviewer'

In [ ]:
# deploy reviewer invitation
openreview.tools.recruit_reviewer(client, rev_email, rev_firstname,
                                 hash_seed = icml.HASH_SEED,
                                 recruit_reviewers_id = icml.RECRUIT_REVIEWERS_ID,
                                 recruit_message = icml.RECRUIT_REVIEWERS_MESSAGE,
                                 recruit_message_subj = icml.RECRUIT_MESSAGE_SUBJ,
                                 reviewers_invited_id = icml.REVIEWERS_INVITED_ID
                                 )

### Double-blind Paper Submission

In [ ]:
# post the submission and blind submission invitations
posted_submission_inv = client.post_invitation(icml.submission_inv)
posted_blind_inv = client.post_invitation(icml.blind_submission_inv)

The following section is for the ICML demo.

It collects papers from ICLR 2018 and posts them as if they were ICML submissions.

In [ ]:
# collect old ICLR papers and post them
old_iclr_papers = client.get_notes(invitation='ICLR.cc/2018/Conference/-/Submission', limit=20)
print len(old_iclr_papers)

In [ ]:
# post the ICLR papers
for paper in old_iclr_papers:
    paper.writers.remove('ICLR.cc/2018/Conference')
    paper.writers.append('ICML.cc/2019/Conference')
    if 'TL;DR' in paper.content:
        paper.content.pop('TL;DR')
    
    posted_submission = client.post_note(openreview.Note(**{
        'writers': paper.writers,
        'readers': [icml.CONFERENCE_ID] + paper.content['authorids'] + paper.signatures,
        'content': paper.content,
        'invitation': icml.SUBMISSION_ID
    }))
    print posted_submission.content['title']
    

### Area Chair and Reviewer bidding

In [ ]:
# OpenReview lets area chairs and reviewers (and whoever else) bid on papers.
icml.add_bid.invitees = [icml.REVIEWERS_ID, icml.AREA_CHAIRS_ID]
client.post_invitation(icml.add_bid)

### Reviewer recommendations from area chairs

In [ ]:
# get all currently submitted papers
icml_submissions = openreview.tools.get_all_notes(client, icml.BLIND_SUBMISSION_ID)

# Add reviewer rec tag invitation to the last five papers
for paper in icml_submissions[0:5]:
    rec_inv = openreview.invitations.from_template(icml.ac_recommendation_template, paper)
    rec_inv.invitees = ['icmlareachair@openreview.net']
    posted_inv = client.post_invitation(rec_inv)
    print posted_inv.id

In [ ]:
# convert reviewers to canonical identities

reviewers_group = client.get_group(icml.REVIEWERS_ID)
for member in reviewers_group.members:
    if '@' in member:
        profile = client.get_profile(member)
        client.remove_members_from_group(reviewers_group, member)
        client.add_members_to_group(reviewers_group, profile.id)
        

### Official Reviews, Official Comments, and Open Comments

In [ ]:
# get all currently submitted papers
icml_submissions = openreview.tools.get_all_notes(client, icml.BLIND_SUBMISSION_ID)

# assign some papers
for paper in icml_submissions[0:5] + [icml_submissions[-1]]:
    openreview.tools.assign(client, paper.number, icml.CONFERENCE_ID,
        reviewer_to_add = 'icmlreviewer@openreview.net',
        parent_label = 'Reviewers',
        individual_label = 'AnonReviewer',
        individual_group_params = {
            'readers': [icml.AREA_CHAIRS_ID]
        })
    openreview.tools.assign(client, paper.number, icml.CONFERENCE_ID,
        reviewer_to_add = 'icmlareachair@openreview.net',
        parent_label = 'Area_Chairs',
        individual_label = 'Area_Chair')

In [ ]:
# get all currently submitted papers
icml_submissions = openreview.tools.get_all_notes(client, icml.BLIND_SUBMISSION_ID)

# enable reviews
for paper in icml_submissions:
    client.post_invitation(openreview.invitations.from_template(icml.official_review_template, paper))
    client.post_invitation(openreview.invitations.from_template(icml.official_comment_template, paper))
    client.post_invitation(openreview.invitations.from_template(icml.meta_review_template, paper))

In [ ]:
# get all currently submitted papers
icml_submissions = openreview.tools.get_all_notes(client, icml.BLIND_SUBMISSION_ID)

for paper in icml_submissions:
    metadata_params = {
        'forum': paper.id,
        'replyto': paper.id,
        'invitation': icml.REVIEWER_METADATA_ID,
        'readers': {
            'values': [
                icml.CONFERENCE_ID
            ]
        },
        'writers': {
            'values': [icml.CONFERENCE_ID]
        },
        'signatures': {
            'values': [icml.CONFERENCE_ID]},
        'content': {
            'groups': {
                icml.REVIEWERS_ID: [
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Red_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Blue_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~Green_Reviewer1'
                    },
                    {
                        'scores': {'affinity': 1.0},
                        'constraints': {'cmtConflict': '-inf'},
                        'userId': '~ICML_Reviewer1'
                    }
                ]
            }
        }
    }

In [ ]:
import random

client.post_invitation(icml.assignment_inv)

for paper in icml_submissions:
    finalscore = random.random()
    assignment_params = {
        'invitation': icml.ASSIGNMENT_INV_ID,
        'readers': [icml.CONFERENCE_ID],
        'writers': [icml.CONFERENCE_ID],
        'signatures': [icml.CONFERENCE_ID],
        'forum': paper.id,
        'replyto': paper.id,
        'content': {
            'label': 'reviewers',
            'assignedGroups': [
                {
                    "finalScore": 1.0,
                    "scores": {
                        "affinity": 1.0
                    },
                    "constraints": {},
                    "userId": "~Red_Reviewer1"
                },
            ],
            'alternateGroups': [
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {'conflict': '-inf'},
                    "userId": "~Blue_Reviewer1"
                },
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {},
                    "userId": "~Green_Reviewer1"
                },
                {
                    "finalScore": random.random(),
                    "scores": {
                        "affinity": random.random()
                    },
                    "constraints": {},
                    "userId": "~ICML_Reviewer1"
                },
            ]
        }   
    }
    
    client.post_note(openreview.Note(**assignment_params))


In [ ]:
client.post_invitation(icml.config_inv)
config_note = client.post_note(openreview.Note(**{
    'invitation': icml.CONFIG_INV_ID,
    'readers': [icml.CONFERENCE_ID],
    'writers': [icml.CONFERENCE_ID],
    'signatures': [icml.CONFERENCE_ID],
    'content': {
        'label': 'reviewers',
        'configuration': {
            'minusers': 1,
            'maxusers': 1,
            'minpapers': 0,
            'maxpapers': 8,
            'alternates': 5,
            'weights': {
                'affinity': 1.0,
                'acRecommendation': 3.0,
                'bidScore': 1.0
            }
        },
        'constraints': {},
        'paper_invitation': icml.BLIND_SUBMISSION_ID,
        'metadata_invitation': icml.REVIEWER_METADATA_ID,
        'assignment_invitation': icml.ASSIGNMENT_INV_ID,
        'match_group': icml.REVIEWERS_ID,
        'status': 'complete' # 'queued', 'processing', 'complete' or 'error'
    }
}))